# Phase-field crystal tutorail: Polycrystalline materials

In this tutorial, we will look at different ways of creating polycrystalline materials with the PFC framework.

In [ ]:
!pip install comfit -q

The first way to create a polycrystalline material is to quench the PFC. 

In [ ]:

import comfit as cf
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt


In [ ]:
pfc = cf.PhaseFieldCrystal2DTriangular(30, 23)

#Set a random initial condition
pfc.psi = 0.01*(np.random.rand(pfc.xRes,pfc.yRes) - 0.5) + pfc.psi0
pfc.psi_f = sp.fft.fftn(pfc.psi)

pfc.plot_field(pfc.psi)

In [ ]:
pfc.evolve_PFC(3000)
pfc.plot_field(pfc.psi)

Increasing the size of the simulation will create a larger polycrystalline material.

However, in many situations we want to control the polycrystalline material. 
In this case, we want to prescribe the regions with the different lattice orientations. 
Let's see how that can be done

We start by creating a simple inclusion, which is rotated with respect to the rest of the material.
To easily see what is going on, we will use the square PFC model.

In [ ]:
pfc = cf.PhaseFieldCrystal2DSquare(40, 40)

# This creates a standard orientation of the crystal
pfc.conf_PFC_from_amplitudes()


psi_rotated = pfc.calc_PFC_from_amplitudes(rotation=np.pi/3)
pfc.plot_field(psi_rotated)


Now we just need to paste this region in the center onto the old field contained in the variable `pfc.psi`.

In [ ]:
# Specify the region centered at the mid position with radius 5 a0.
inclusion_region = pfc.calc_region_disk(pfc.rmid, 5*pfc.a0)

# Set the rotated field in the inclusion region
pfc.psi[inclusion_region] = psi_rotated[inclusion_region]
pfc.psi_f = sp.fft.fftn(pfc.psi)

pfc.plot_field(pfc.psi)

The interface is very sharp. In order to smooth it, we simply evolve the field for a few time steps.

NB! Remember that in order for this method to work, we need to also specify the fourier transform of the field, which is why we have added the `pfc.psi_f = sp.fft.fftn(pfc.psi)` line above. 

In [ ]:
pfc.evolve_PFC(100)

pfc.plot_field(pfc.psi)

Let create a bigger inclusion and create an evolution gif. 

In [ ]:
pfc = cf.PhaseFieldCrystal2DSquare(40, 40)

# This creates a standard orientation of the crystal
pfc.conf_PFC_from_amplitudes()

# Create the rotated field
psi_rotated = pfc.calc_PFC_from_amplitudes(rotation=np.pi/6)

# Specify the region centered at the mid position with radius 5 a0.
inclusion_region = pfc.calc_region_disk(pfc.rmid, 10*pfc.a0)

# Set the rotated field in the inclusion region
pfc.psi[inclusion_region] = psi_rotated[inclusion_region]
pfc.psi_f = sp.fft.fftn(pfc.psi)

for n in range(100):
    pfc.evolve_PFC(100)
    pfc.plot_field(pfc.psi)
    cf.tool_save_plot(n)
cf.tool_make_animation_gif(n,'Inclusion_PFC_normal')

This simulation used the normal PFC evolution equation. 
It hardly moves at all, because the inclusion is quite stuck.
Zooming in however, reveals how it is slowly moving.

If we instead employ the hydrodynamic model, we get a much quicker evolution of the interface.

In [ ]:
pfc = cf.PhaseFieldCrystal2DSquare(40, 40)

# This creates a standard orientation of the crystal
pfc.conf_PFC_from_amplitudes()

# Create the rotated field
psi_rotated = pfc.calc_PFC_from_amplitudes(rotation=np.pi/6)

# Specify the region centered at the mid position with radius 5 a0.
inclusion_region = pfc.calc_region_disk(pfc.rmid, 10*pfc.a0)
pfc.evolve_PFC(100) #Smooth the interface

# Set the rotated field in the inclusion region
pfc.psi[inclusion_region] = psi_rotated[inclusion_region]
pfc.psi_f = sp.fft.fftn(pfc.psi)

for n in range(100):
    pfc.evolve_PFC_hydrodynamic(100)
    pfc.plot_field(pfc.psi[0])
    cf.tool_save_plot(n)
cf.tool_make_animation_gif(n,'Inclusion_PFC_hydrodynamic')